In [1]:
import itertools
import timeit
import numpy as np

In [2]:
%load_ext line_profiler

This is the original function, as given in the exercise. The goal is to assess it with line_profiler and to write a faster version.

In [3]:
def primes(n=1000): 
    A = [True] * (n+1)
    
    A[0] = False
    A[1] = False
    for i in range(2, int(n**0.5)):
        if A[i]:
            for j in range(i**2, n+1, i):
                A[j] = False

    return [x for x in range(2, n) if A[x]]

In [4]:
%lprun -f primes primes(10000)

Based on line profiler, I can see that the function spends most of its time generating the range of j values and setting A[j] to false. The list comprehension at the end of the function also makes a small contribution. I wasn't able to make a significant improvement on my own, so here is an optimized version from the internet. 

In [5]:
def faster_primes(n=1000):
    
    bools = np.array(([True] * (n+1)))
    np.put(bools, range(2), False)
    
    for i in range(2, int(n**0.5)):
        if bools[i]:
            bools[i*i::i] = False
            
    return list(itertools.compress(range(n), bools))

Here is the optimized version's line profiler. 

In [6]:
%lprun -f faster_primes faster_primes(10000)

Here is the testing script given in the assignment. 

In [7]:
from nose.tools import assert_equal, assert_less
assert_equal(primes(1000), faster_primes(1000))
assert_less(timeit.timeit(faster_primes), timeit.timeit(primes))

Here is my timeit since I wanted to see the speedup. 

In [8]:
num_tests = 200
num_repeats = 10

time_vector_original = timeit.repeat("primes(10000)", number = num_tests, repeat = num_repeats, globals=globals())
time_best_original = min(time_vector_original)
print("Original Function, Best Time: {:.3f}".format(time_best_original))

time_vector_opt = timeit.repeat("faster_primes(10000)", number = num_tests, repeat = num_repeats, globals=globals())
time_best_opt = min(time_vector_opt)
print("Optimized Function, Best Time: {:.3f}".format(time_best_opt))

speedup_opt = time_best_original / time_best_opt

print("Optimized Function, Speedup: {:.2f}x".format(speedup_opt))

Original Function, Best Time: 0.239
Optimized Function, Best Time: 0.132
Optimized Function, Speedup: 1.81x
